In [7]:
%%capture
import sys

is_in_colab = 'google.colab' in sys.modules

if is_in_colab:
  from google.colab import drive
  drive.mount('/content/drive')

  %cd /content/drive/MyDrive/KU_NLP
  !pip install -r requirements.txt

# 2. Representation Learning

In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from enum import Enum

from models.answer_exists_models import *

import fasttext
import fasttext.util

In [8]:
!pip install compress-fasttext

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 24.0/24.0 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 983.8/983.8 kB 3.7 MB/s eta 0:00:00
  Created wheel for compress-fasttext: filename=compress_fasttext-0.1.3-py3-none-any.whl size=14583 sha256=6baef1fcf882eef53ebae0c1d8679020ee9826de74df8d8ef799cc3c3b06f81a
  Stored in directory: c:\users\rasmu\appdata\local\pip\cache\wheels\c7\63\9f\39db0410175167cee5eeae4fde2405d957cd05c1d8811a51cf
Successfully built compress-fasttext


In [49]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [50]:
class Annotation_error(Enum):
    UNANSWERED = -1
    BAD_TOKENIZATION_OR_DATA = -2
    IGNORED = -3

In [51]:
path_train_set = "data/train_set_stanza.pkl"
path_validation_set = "data/validation_set_stanza.pkl"

In [52]:
train_set = pd.read_pickle(path_train_set)
validation_set = pd.read_pickle(path_validation_set)

In [53]:
train_en = train_set[train_set["language"] == "english"]
train_fi = train_set[train_set["language"] == "finnish"]
train_ja = train_set[train_set["language"] == "japanese"]

validation_en = validation_set[validation_set["language"] == "english"]
validation_fi = validation_set[validation_set["language"] == "finnish"]
validation_ja = validation_set[validation_set["language"] == "japanese"]

# 2a.

In [59]:
train_en

,language,question,document_title,document,document_answer_region
0,english,"[When, was, quantum, field, theory, developed, ?]","[Quantum, field, theory]","[Quantum, field, theory, naturally, began, wit...","(26, 26)"
1,english,"[Who, was, the, first, Nobel, prize, winner, f...","[List, of, Nobel, laureates, in, Literature]","[The, Nobel, Prize, in, Literature, (, Swedish...","(12, 13)"
2,english,"[When, is, the, dialectical, method, used, ?]",[Dialectic],"[Dialectic, or, dialectics, (, Greek, :, διαλε...","(27, 49)"
3,english,"[Who, invented, Hangul, ?]","[Origin, of, Hangul]","[Hangul, was, personally, created, and, promul...","(16, 18)"
4,english,"[What, do, Grasshoppers, eat, ?]",[Grasshopper],"[Grasshoppers, are, plant, -, eaters, ,, with,...","(1, 37)"
...,...,...,...,...,...
7384,english,"[What, was, Neil, Brooks, ', fastest, recorded...","[Swimming, at, the, 1980, Summer, Olympics, –,...","[The, medley, relay, was, scheduled, in, the, ...",Annotation_error.UNANSWERED
7385,english,"[Who, are, the, three, most, important, easter...","[Eastern, philosophy]","[Sāmkhya, is, a, dualist, philosophical, tradi...",Annotation_error.UNANSWERED
7386,english,"[Who, was, costume, designer, for, the, first,...","[John, Mollo]","[Mollo, was, surprised, by, the, success, of, ...",Annotation_error.UNANSWERED
7387,english,"[Who, developed, the, first, thermonuclear, we...","[History, of, nuclear, weapons]","[In, the, end, ,, President, Truman, made, the...",Annotation_error.UNANSWERED


In [46]:
# If the question is answered, then predict "1". Otherwise predict "0".
def get_target(data):
    answer_set = data['document_answer_region']
    y = np.empty(answer_set.shape[0])

    for i, answer in enumerate(answer_set):
        if type(answer) == Annotation_error and answer == Annotation_error.UNANSWERED: # @TODO: if we don't do the annotation stuff, then we can check for -1 here
            y[i] = 0
        else:
            y[i] = 1
    return y

In [43]:
import compress_fasttext

In [44]:
tok2vec = compress_fasttext.models.CompressedFastTextKeyedVectors.load("fasttext-en-mini")

In [102]:
tok2vec

In [90]:
question_ids = words_to_ids(train_en['question'])
lengths = [len(question_tokens) for question_tokens in train_en['question']]
max_length = max(lengths)

In [60]:
help(tok2vec)

Help on CompressedFastTextKeyedVectors in module compress_fasttext.compress object:

class CompressedFastTextKeyedVectors(gensim.models.fasttext.FastTextKeyedVectors)
 |  CompressedFastTextKeyedVectors(*args, **kwargs)
 |  
 |  This class extends FastTextKeyedVectors by fixing several issues:
 |  - index2word of a freshly created model is initialized from its vocab
 |  - the model does not keep heavy and useless vectors_ngrams_norm
 |  - word_vec() method with use_norm applies normalization in the right place
 |  
 |  Method resolution order:
 |      CompressedFastTextKeyedVectors
 |      gensim.models.fasttext.FastTextKeyedVectors
 |      gensim.models.keyedvectors.KeyedVectors
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Vectors and vocab for :class:`~gensim.models.fasttext.FastText`.
 |      
 |      Implements significant parts of the FastText algorithm.  For example,
 |      the :func:`word_ve

In [ ]:
X_train = np.empty()
for i in rangetrain_en['question']

In [56]:
y_train = get_target(train_en)

In [57]:
y_train

array([1., 1., 1., ..., 0., 0., 0.])

In [287]:
import torch
from torch import nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [126]:
class TinyNetwork(nn.Module):

    def __init__(self, pretrained_embeddings: torch.tensor, n_classes: int = 2):
        super(TinyNetwork, self).__init__()

        self.net = nn.Sequential(
            nn.Embedding.from_pretrained(pretrained_embeddings),
            nn.Linear(pretrained_embeddings.shape[1], n_classes),
            nn.Softmax(dim=0)
        )
        
    def forward(self, inputs):
        return self.net.forward(inputs)

In [657]:
class TinyNetwork2(nn.Module):

    def __init__(self, pretrained_embeddings: torch.tensor, lstm_dim: int = 6, n_classes: int = 2):
        super(TinyNetwork2, self).__init__()

        self.lstm_dim = lstm_dim
        self.n_classes = n_classes
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings)
        self.biLSTM = nn.LSTM(pretrained_embeddings.shape[1], lstm_dim, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(2*lstm_dim, n_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, inputs, input_lengths):
        # b x sl x emb_dim
        embeds = self.embedding(inputs)
    
        # Pack padded: This is necessary for padded batches input to an RNN
        lstm_in = nn.utils.rnn.pack_padded_sequence(embeds, input_lengths, batch_first=True, enforce_sorted=False)
        
        # Pass the packed sequence through the BiLSTM
        lstm_out, _hidden = self.biLSTM(lstm_in)

        # Unpack the packed sequence (b x sl x 2*lstm_dim)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # representation of the last lstm unit (b x 2*lstm_dim)
        n_batch = lstm_out.shape[0]
        input_ends = input_lengths-1
        ff_in = torch.vstack([lstm_out[i, input_ends[i], :] for i in range(n_batch)])
        # Alternative: Max element-wise over all hidden units
        # ff_in = torch.max(lstm_out, 1)[0]
        
        # Get logits (b x n_classes)
        logits = self.linear(ff_in)
        
        return self.softmax(logits)

In [658]:
# b1: [.1, .4, .6], [.!, .!, .!], [.0, .0, .0], [.0, .0, .0],
# b2: [.1, .4, .6], [.1, .4, .6], [.!, .!, .!], [.0, .0, .0],
# b3: [.1, .4, .6], [.1, .4, .6], [.1, .4, .6], [.!, .!, .!],

In [659]:
a = torch.tensor([[[1,2],[2,3],[6,4],[4,5]], [[5,6],[6,7],[7,8],[8,9]]])
indices = [0, 2]
torch.vstack([a[i,indices[i],:] for i in range(len(indices))])

tensor([[1, 2],
        [7, 8]])

In [664]:
pretrained_embeddings = torch.tensor(tok2vec.get_normed_vectors(), dtype=torch.float32)
model = TinyNetwork2(pretrained_embeddings)
print(model)

TinyNetwork2(
  (embedding): Embedding(20000, 300)
  (biLSTM): LSTM(300, 6, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=12, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)


In [665]:
# Test the forward pass with dummy data
inputs = torch.tensor([[42, 1, 8, 0]], dtype=torch.int)
input_lengths = torch.tensor([3], dtype=torch.int)
out = model.forward(inputs, input_lengths)

print("Output shape:", out.size())
print(f"Output probabilities:\n{out.detach().numpy()}")

Output shape: torch.Size([1, 2])
Output probabilities:
[[0.5093861  0.49061394]]


In [666]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [669]:
num_epochs = 10

train_sentences = torch.tensor([[42, 1, 0], [42, 1, 0], [18, 14, 0], [18, 1, 2]], dtype=torch.int32)
train_lengths = torch.tensor([2, 2, 2, 3], dtype=torch.int32)
train_target = torch.tensor([1, 1, 0, 0])

step = 0
model.train()

train_accuracies = []
valid_accuracies = []

correct = 0
total = 0
        
for epoch in range(num_epochs):
    
    correct = 0
    total = 0
    
    inputs = train_sentences
    lengths = train_lengths
    targets = train_target
    
    # Forward pass.
    outputs = model(inputs, lengths)
    loss = loss_fn(outputs, targets)
    
    # Backward pass.
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    step += 1

    # Compute accuracy.
    predictions = outputs.max(1)[1]
    
    #print(outputs)
    #print(predictions)
    
    print("accuracy: {}".format(accuracy_score(targets, predictions)))
        

print("Finished training.")


accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
accuracy: 1.0
Finished training.


# Prepare data

# Model

In [19]:
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple
import torch
from torch import nn

In [20]:
def check_word_id(token, ft):
    word_id = ft.get_word_id(token)
    assert word_id != -1, "OOV token found -> {}".format(token)
    return word_id

In [21]:
def text_to_batch_bilstm(text: List, ft, max_len=512) -> Tuple[List, List]:
    """
    Creates a tokenized batch for input to a bilstm model
    :param text: A list of sentences to tokenize
    :param tokenizer: A tokenization function to use (i.e. fasttext)
    :return: Tokenized text as well as the length of the input sequence
    """
    # Some light preprocessing
    #input_ids = [tokenizer.encode_ids_with_eos(t)[:max_len] for t in text]
    input_ids = [[check_word_id(token, ft) for token in tokens] for tokens in text]
    
    return input_ids, [len(ids) for ids in input_ids]

In [22]:
text_to_batch_bilstm([["sailor", "sailor"], ["where"], ["Where"]], ft)

([[21180, 21180], [141], [1944]], [2, 1, 1])

In [23]:
def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Combines multiple data samples into a single batch
    :param input_data: The combined input_ids, seq_lens, and labels for the batch
    :return: A tuple of tensors (input_ids, seq_lens, labels)
    """
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    # Pad all of the input samples to the max length (25000 is the ID of the [PAD] token)
    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    # Make sure each sample is max_length long
    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)

In [24]:
# Define the model
class BiLSTMNetwork(nn.Module):
    """
    Basic BiLSTM network
    """
    def __init__(
            self,
            pretrained_embeddings: torch.tensor,
            lstm_dim: int,
            dropout_prob: float = 0.1,
            n_classes: int = 2
    ):
        """
        Initializer for basic BiLSTM network
        :param pretrained_embeddings: A tensor containing the pretrained BPE embeddings
        :param lstm_dim: The dimensionality of the BiLSTM network
        :param dropout_prob: Dropout probability
        :param n_classes: The number of output classes
        """

        # First thing is to call the superclass initializer
        super(BiLSTMNetwork, self).__init__()

        # We'll define the network in a ModuleDict, which makes organizing the model a bit nicer
        # The components are an embedding layer, a 2 layer BiLSTM, and a feed-forward output layer
        self.model = nn.ModuleDict({
            'embeddings': nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1),
            'bilstm': nn.LSTM(
                pretrained_embeddings.shape[1],
                lstm_dim,
                1,
                batch_first=True,
                dropout=dropout_prob,
                bidirectional=True),
            'cls': nn.Linear(2*lstm_dim, n_classes)
        })
        self.n_classes = n_classes
        self.dropout = nn.Dropout(p=dropout_prob)

        # Initialize the weights of the model
        self._init_weights()

    def _init_weights(self):
        all_params = list(self.model['bilstm'].named_parameters()) + \
                     list(self.model['cls'].named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens, labels = None):
        """
        Defines how tensors flow through the model
        :param inputs: (b x sl) The IDs into the vocabulary of the input samples
        :param input_lens: (b) The length of each input sequence
        :param labels: (b) The label of each sample
        :return: (loss, logits) if `labels` is not None, otherwise just (logits,)
        """

        # Get embeddings (b x sl x edim)
        embeds = self.model['embeddings'](inputs)
        
        
        # Pack padded: This is necessary for padded batches input to an RNN
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        # Pass the packed sequence through the BiLSTM
        lstm_out, hidden = self.model['bilstm'](lstm_in)

        # Unpack the packed sequence --> (b x sl x 2*lstm_dim)
        lstm_out,_ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # Max pool along the last dimension
        ff_in = self.dropout(torch.max(lstm_out, 1)[0])
        # Some magic to get the last output of the BiLSTM for classification (b x 2*lstm_dim)
        #ff_in = lstm_out.gather(1, input_lens.view(-1,1,1).expand(lstm_out.size(0), 1, lstm_out.size(2)) - 1).squeeze()

        # Get logits (b x n_classes)
        logits = self.model['cls'](ff_in).view(-1, self.n_classes)
        outputs = (logits,)
        if labels is not None:
            # Xentropy loss
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            outputs = (loss,) + outputs

        return outputs

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
pretrained_embeddings = np.concatenate((np.zeros((1,pretrained_embeddings.shape[1])),pretrained_embeddings), axis=0)
pretrained_embeddings.shape

In [21]:
# Create the model
model = BiLSTMNetwork(
    pretrained_embeddings=torch.FloatTensor(pretrained_embeddings), 
    lstm_dim=lstm_dim, 
    dropout_prob=0.1, 
    n_classes=2
  ).to(device)

NameError: name 'pretrained_embeddings' is not defined

# 2b.

In [ ]:
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
ft.get_dimension()
ft.get_word_vector('king').shape

fasttext.util.reduce_model(ft, 80)
ft.get_dimension()

"asdasdsad" in ft.words
ft.get_nearest_neighbors('cookie')

In [1]:
import spacy

In [2]:
nlp = spacy.load('en_core_web_md')

In [30]:
nlp.vocab["old"].vector.shape

(300,)

In [4]:
help(nlp)

Help on English in module spacy.lang.en object:

class English(spacy.language.Language)
 |  English(vocab: Union[spacy.vocab.Vocab, bool] = True, *, max_length: int = 1000000, meta: Dict[str, Any] = {}, create_tokenizer: Union[Callable[[ForwardRef('Language')], Callable[[str], spacy.tokens.doc.Doc]], NoneType] = None, batch_size: int = 1000, **kwargs) -> None
 |  
 |  A text-processing pipeline. Usually you'll load this once per process,
 |  and pass the instance around your application.
 |  
 |  Defaults (class): Settings, data and factory methods for creating the `nlp`
 |      object and processing pipeline.
 |  lang (str): IETF language code, such as 'en'.
 |  
 |  DOCS: https://spacy.io/api/language
 |  
 |  Method resolution order:
 |      English
 |      spacy.language.Language
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  Defaults = <class 'spacy.lang.en.EnglishDefaults'>
 |      Language data defaults, available via Language.Defaults. Can be


https://github.com/pytorch/text/issues/1350
https://christopher5106.github.io/deep/learning/2020/04/02/fasttext_pretrained_embeddings_subword_word_representations.html

In [37]:
import compress_fasttext
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from compress_fasttext.feature_extraction import FastTextTransformer

small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load('fasttext-en-mini')

classifier = make_pipeline(
    FastTextTransformer(model=small_model), 
    LogisticRegression()
).fit(
    ['banana', 'soup', 'burger', 'car', 'tree', 'city'],
    [1, 1, 1, 0, 0, 0]
)
classifier.predict(['jet', 'train', 'cake', 'apple'])
# array([0, 0, 1, 1])

array([0, 0, 1, 1])

In [38]:
help(small_model)

Help on CompressedFastTextKeyedVectors in module compress_fasttext.compress object:

class CompressedFastTextKeyedVectors(gensim.models.fasttext.FastTextKeyedVectors)
 |  CompressedFastTextKeyedVectors(*args, **kwargs)
 |  
 |  This class extends FastTextKeyedVectors by fixing several issues:
 |  - index2word of a freshly created model is initialized from its vocab
 |  - the model does not keep heavy and useless vectors_ngrams_norm
 |  - word_vec() method with use_norm applies normalization in the right place
 |  
 |  Method resolution order:
 |      CompressedFastTextKeyedVectors
 |      gensim.models.fasttext.FastTextKeyedVectors
 |      gensim.models.keyedvectors.KeyedVectors
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Vectors and vocab for :class:`~gensim.models.fasttext.FastText`.
 |      
 |      Implements significant parts of the FastText algorithm.  For example,
 |      the :func:`word_ve

In [70]:
small_model.get_normed_vectors().shape

(20000, 300)

In [71]:
small_model.get_index("slap")

15784

In [82]:
small_model.most_similar("quantum")

[('Quantum', 0.5923356945161),
 ('physics', 0.4987263608889812),
 ('computational', 0.4833229306372649),
 ('cosmic', 0.46287664812730667),
 ('atomic', 0.4555259364535978),
 ('atoms', 0.4543258391303013),
 ('electron', 0.4415847215404407),
 ('electromagnetic', 0.4342020722504953),
 ('optical', 0.4341506741975586),
 ('physicist', 0.43388256332181196)]

In [83]:
small_model["sailor"].shape

(300,)